In [20]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd
import time
import torch
from huggingface_hub import login

# Giriş yap (gerekiyorsa)
# login(token="HF_TOKENINIZ")

# Yeni model seçimi
model_name = "facebook/nllb-200-distilled-600M"  # 600 milyon parametreli versiyon
# model_name = "facebook/nllb-200-1.3B"  # Daha büyük versiyon (1.3 milyar parametre)

# Model ve tokenizer yükleme
print(f"{model_name} modeli yükleniyor...")
start_load = time.time()

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

print(f"Model yükleme süresi: {time.time()-start_load:.2f}s")
print(f"Model boyutu: {model.num_parameters()/1e6:.1f}M parametre")
print(f"Kullanılan cihaz: {device}")

# Çeviri fonksiyonu (İngilizce → Türkçe)
def translate_to_turkish(text):
    # NLLB'de Türkçe için dil kodu: 'tur_Latn'
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

    # NLLB özel parametreleri
    outputs = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids("tur_Latn")  # Düzeltme burada
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)
# Test cümlesi
test_text = "This is a sample sentence to test the translation model."
print("\nTest çevirisi:")
print(f"Orijinal: {test_text}")
print(f"Çeviri: {translate_to_turkish(test_text)}")

# Veri setini yükleme ve çeviri
df = pd.read_csv('test_dataset_500_sentences.csv')  # Aynı veri seti
total_sentences = len(df)
print(f"\nToplam {total_sentences} cümle çevirisi başlatılıyor...")

results = []
start_time = time.time()

for i, sentence in enumerate(df['Sentence'], 1):
    start = time.time()
    translation = translate_to_turkish(sentence)
    elapsed = time.time() - start
    results.append({'orijinal': sentence, 'çeviri': translation, 'süre': elapsed})

    if i % 50 == 0:
        print(f"{i}/{total_sentences} cümle çevrildi | Son cümle süresi: {elapsed:.3f}s")

total_time = time.time() - start_time
print(f"\nToplam süre: {total_time:.2f}s")
print(f"Ortalama süre/cümle: {total_time/total_sentences:.3f}s")

# Sonuçları kaydet
results_df = pd.DataFrame(results)
results_df.to_csv(f'nllb_results_{model_name.split("/")[-1]}.csv', index=False, encoding='utf-8-sig')
print("Çeviriler kaydedildi.")

facebook/nllb-200-distilled-600M modeli yükleniyor...
Model yükleme süresi: 6.51s
Model boyutu: 615.1M parametre
Kullanılan cihaz: cuda

Test çevirisi:
Orijinal: This is a sample sentence to test the translation model.
Çeviri: Bu, çeviri modelini test etmek için örnek bir cümle.

Toplam 500 cümle çevirisi başlatılıyor...
50/500 cümle çevrildi | Son cümle süresi: 0.473s
100/500 cümle çevrildi | Son cümle süresi: 0.250s
150/500 cümle çevrildi | Son cümle süresi: 0.219s
200/500 cümle çevrildi | Son cümle süresi: 0.443s
250/500 cümle çevrildi | Son cümle süresi: 0.247s
300/500 cümle çevrildi | Son cümle süresi: 0.296s
350/500 cümle çevrildi | Son cümle süresi: 0.249s
400/500 cümle çevrildi | Son cümle süresi: 0.365s
450/500 cümle çevrildi | Son cümle süresi: 0.229s
500/500 cümle çevrildi | Son cümle süresi: 0.271s

Toplam süre: 149.93s
Ortalama süre/cümle: 0.300s
Çeviriler kaydedildi.


In [25]:
from huggingface_hub import HfApi, ModelInfo
import os

# Model bilgisi
model_name = "facebook/nllb-200-distilled-600M"
api = HfApi()

# 1. Model repo bilgilerini al
try:
    model_info = api.model_info(model_name)
    files = model_info.siblings  # Repodaki tüm dosyalar

    print(f"\n=== {model_name} Model Dosyaları ===")
    total_size = 0

    for file in files:
        size_mb = file.size / (1024 * 1024)
        print(f"{file.rfilename.ljust(40)}: {size_mb:>7.2f} MB")
        total_size += file.size

    print(f"\nToplam Boyut: {total_size/(1024*1024):.2f} MB")

except Exception as e:
    print(f"Hata oluştu: {str(e)}")

# 2. Lokal cache boyutunu kontrol et
from transformers import TRANSFORMERS_CACHE
cache_dir = os.path.join(TRANSFORMERS_CACHE, f"models--{model_name.replace('/', '--')}")

if os.path.exists(cache_dir):
    print("\n=== Lokal Cache Boyutu ===")
    total_local = 0
    for root, _, files in os.walk(cache_dir):
        for file in files:
            fp = os.path.join(root, file)
            total_local += os.path.getsize(fp)
    print(f"{total_local/(1024*1024):.2f} MB")
else:
    print("\nLokal cache bulunamadı!")


=== facebook/nllb-200-distilled-600M Model Dosyaları ===
Hata oluştu: unsupported operand type(s) for /: 'NoneType' and 'int'

=== Lokal Cache Boyutu ===
9428.03 MB


In [26]:
!pip install pynvml
import pynvml

def monitor_gpu():
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)

    print("\n=== GPU Bilgileri ===")
    # Model yüklenmeden önce
    pre_mem = pynvml.nvmlDeviceGetMemoryInfo(handle).used/1024**2

    # Modeli yükle
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

    # Model yüklendikten sonra
    post_mem = pynvml.nvmlDeviceGetMemoryInfo(handle).used/1024**2
    print(f"GPU Bellek Kullanımı:")
    print(f"  Model yükleme öncesi: {pre_mem:.2f} MB")
    print(f"  Model yükleme sonrası: {post_mem:.2f} MB")
    print(f"  Modelin VRAM kullanımı: {post_mem-pre_mem:.2f} MB")

    # Çeviri sırasında maksimum kullanım
    max_mem = pre_mem
    test_text = "This is a long sentence to test memory usage during translation. " * 10
    for _ in range(10):
        _ = translate_to_turkish(test_text)
        current_mem = pynvml.nvmlDeviceGetMemoryInfo(handle).used/1024**2
        max_mem = max(max_mem, current_mem)

    print(f"  Maksimum gözlemlenen kullanım: {max_mem:.2f} MB")
    pynvml.nvmlShutdown()

monitor_gpu()


=== GPU Bilgileri ===
GPU Bellek Kullanımı:
  Model yükleme öncesi: 5153.88 MB
  Model yükleme sonrası: 5153.88 MB
  Modelin VRAM kullanımı: 0.00 MB
  Maksimum gözlemlenen kullanım: 5205.88 MB
